# Práctica 4: Planificación automática
# Inteligencia Artificial
# Grado en Ingeniería Informática - Ingeniería del Software
# Universidad de Sevilla

[PDDL](https://planning.wiki/) (_Planning Domain Definition Language_) es una familia de lenguajes que se ha convertido en el estándar para la especificación de dominios y problemas de planificación automática. A medida que este subcampo de la Inteligencia Artificial ha evolucionado, así lo ha hecho también el lenguaje usado para describir los problemas que estudia. Hoy en día hay, por tanto, varias versiones disponibles de PDDL, con diferentes niveles de expresividad.

La [versión 1.2](https://planning.wiki/ref/pddl) de PDDL establece la sintaxis más básica que debe ser entendible por cualquier planificador y es a la que nos referiremos al hablar simplemente de PDDL y la que usaremos en esta práctica.

Un problema de planificación automática se especifica en PDDL en dos partes: por un lado, se especifica el dominio del problema, que establece todos aquellos aspectos (predicados y acciones) que son independientes de la situación concreta que se está tratando de resolver; por otro lado, se especifica el problema en sí, estableciendo exactamente qué objetos existen, en qué situación inicial se encuentran y a qué situación final se pretende llegar.

La sintaxis mínima para el fichero de especificación del dominio es la siguiente:

```
(define
  (domain <nombre_del_dominio>)
  (:requirements :strips)
  (:predicates
    (<nombre_del_predicado> <argumento_1> ... <argumento_m>)
    ...
  )
  (:action <nombre_de_la_acción>
    :parameters (<argumento_1> ... <argumento_n>)
    :precondition (and
      <precondición>
      ...
    )
    :effect (and
      <adición>
      ...
      (not <borrado>)
      ...
    )
  ...
  )
)
```

Por ejemplo, para el problema de la rueda pinchada se podría representar el dominio en un fichero `dominio_rueda_pinchada.pddl` con el siguiente contenido:

```
(define
  (domain dominio_rueda_pinchada)
  (:requirements :strips)
  (:predicates
    (en ?r ?l)
    (no_en ?r ?l)
  )
  (:action quitar_pinchada
    :parameters ()
    :precondition (and
      (en rueda_pinchada eje)
    )
    :effect (and
      (en rueda_pinchada suelo)
      (no_en rueda_pinchada eje)
      (not (en rueda_pinchada eje))
      (not (no_en rueda_pinchada suelo))
    )
  )
  (:action guardar_pinchada
    :parameters ()
    :precondition (and
      (en rueda_pinchada suelo)
      (no_en rueda_repuesto maletero)
    )
    :effect (and
      (en rueda_pinchada maletero)
      (no_en rueda_pinchada suelo)
      (not (en rueda_pinchada suelo))
      (not (no_en rueda_pinchada maletero))
    )
  )
  (:action sacar_repuesto
    :parameters ()
    :precondition (and
      (en rueda_repuesto maletero)
    )
    :effect (and
      (en rueda_repuesto suelo)
      (no_en rueda_repuesto maletero)
      (not (en rueda_repuesto maletero))
      (not (no_en rueda_repuesto suelo))
    )
  )
  (:action poner_repuesto
    :parameters ()
    :precondition (and
      (en rueda_repuesto suelo)
      (no_en rueda_pinchada eje)
    )
    :effect (and
      (en rueda_repuesto eje)
      (no_en rueda_repuesto suelo)
      (not (en rueda_repuesto suelo))
      (not (no_en rueda_repuesto eje))
    )
  )
)
```

La sintaxis mínima para el fichero de especificación del problema es la siguiente:

```
(define
  (problem <nombre_del_problema>)
  (:domain <dominio_del_problema>)
  (:objects
    <nombre_de_objeto>
    ...
  )
  (:init
    <predicado>
    ...
  )
  (:goal (and
      <objetivo>
      ...
    )
  )
)
```

Por ejemplo, para el problema de la rueda pinchada se podría representar el problema en un fichero `problema_rueda_pinchada.pddl` con el siguiente contenido:

```
(define
  (problem problema_rueda_pinchada)
  (:domain dominio_rueda_pinchada)
  (:objects
    rueda_repuesto rueda_pinchada
    eje maletero suelo
  )
  (:init
    (en rueda_repuesto maletero)
    (no_en rueda_repuesto suelo)
    (no_en rueda_repuesto eje)
    (no_en rueda_pinchada maletero)
    (no_en rueda_pinchada suelo)
    (en rueda_pinchada eje)
  )
  (:goal (and
      (en rueda_repuesto eje)
      (en rueda_pinchada maletero)
    )
  )
)
```

Una vez creados los ficheros especificando el dominio del problema y la instancia concreta de este, para encontrar una solución se debe utilizar un planificador. En esta práctica utilizaremos [pyperplan](https://github.com/aibasel/pyperplan), que implementa varias heurísticas y algoritmos de búsqueda:

In [12]:
%pip install git+https://github.com/remykarem/py2pddl#egg=py2pddl

  Cloning https://github.com/remykarem/py2pddl to c:\users\jargu\appdata\local\temp\pip-install-7_wfpj0a\py2pddl_dad869b50d8749d09db736d3e858b8c8
  Resolved https://github.com/remykarem/py2pddl to commit e8b452320652963574a5b144c17bccf6568e4317
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/81.2 kB ? eta -:--:--
     --------------- ------------------------ 30.7/81.2 kB 1.4 MB/s eta 0:00:01
     ---------------------------------------- 81.2/81.2 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for py2pddl: filename=py2pddl-1.0.0-py3-none-any.whl size=9666 sha256=c5b3f9f7ce1b84c2a048ca74b209efc5c6623ac319aa181f4261f9dd91e4a549
  Stored in directory: C:\Users\jargu\AppData\Local\Temp\pip-ephem-wheel-cache-cr1l_hzx\wheels\f4\d4\22\0a9743216279c1ec86e39c4aa89a4a9f57303589d094800834
  Created whe

  Running command git clone --filter=blob:none --quiet https://github.com/remykarem/py2pddl 'C:\Users\jargu\AppData\Local\Temp\pip-install-7_wfpj0a\py2pddl_dad869b50d8749d09db736d3e858b8c8'

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
%pip install pyperplan
!pyperplan --help

   ---------------------------------------- 0.0/69.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/69.5 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/69.5 kB ? eta -:--:--
   ----- ---------------------------------- 10.2/69.5 kB ? eta -:--:--
   ----------- ---------------------------- 20.5/69.5 kB 131.3 kB/s eta 0:00:01
   ----------------------------------- ---- 61.4/69.5 kB 328.2 kB/s eta 0:00:01
   ----------------------------------- ---- 61.4/69.5 kB 328.2 kB/s eta 0:00:01
   ----------------------------------- ---- 61.4/69.5 kB 328.2 kB/s eta 0:00:01
   ----------------------------------- ---- 61.4/69.5 kB 328.2 kB/s eta 0:00:01
   ---------------------------------------- 69.5/69.5 kB 190.0 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


usage: pyperplan [-h] [-l {debug,info,warning,error}]
                 [-H {blind,landmark,lmcut,hadd,hff,hmax,hsa}]
                 [-s {astar,wastar,gbf,bfs,ehs,ids,sat}]
                 [domain] problem

positional arguments:
  domain
  problem

options:
  -h, --help            show this help message and exit
  -l {debug,info,warning,error}, --loglevel {debug,info,warning,error}
  -H {blind,landmark,lmcut,hadd,hff,hmax,hsa}, --heuristic {blind,landmark,lmcut,hadd,hff,hmax,hsa}
                        Select a heuristic (default: hff)
  -s {astar,wastar,gbf,bfs,ehs,ids,sat}, --search {astar,wastar,gbf,bfs,ehs,ids,sat}
                        Select a search algorithm from astar, weighted astar,
                        greedy best first, breadth first, enforced
                        hillclimbing, iterative deepening, sat solve (default:
                        bfs)


Una vez instalado, basta ejecutarlo desde consola (obsérvese que para hacerlo desde una celda de Jupyter hay que usar el operador !), indicando la heurística y algoritmo a usar y el dominio e instancia del problema. Si el planificador encuentra una solución al problema, entonces la proporciona en un fichero con el mismo nombre que el de la instancia del problema, pero al que se le ha añadido la extensión soln al final.

Buscamos una solución al problema de la rueda pinchada usando el algoritmo $A^{*}$ y la heurística $h^{add}$.

In [6]:
!pyperplan -H hadd -s astar dominio_rueda_pinchada.pddl problema_rueda_pinchada.pddl

2025-04-24 16:07:03,012 INFO     using search: astar_search
2025-04-24 16:07:03,012 INFO     using heuristic: hAddHeuristic
2025-04-24 16:07:03,013 INFO     Parsing Domain c:\Program Files\practicas IA\Práctica_4\Planificación automática\dominio_rueda_pinchada.pddl
2025-04-24 16:07:03,039 INFO     Parsing Problem c:\Program Files\practicas IA\Práctica_4\Planificación automática\problema_rueda_pinchada.pddl
2025-04-24 16:07:03,059 INFO     2 Predicates parsed
2025-04-24 16:07:03,059 INFO     4 Actions parsed
2025-04-24 16:07:03,059 INFO     5 Objects parsed
2025-04-24 16:07:03,059 INFO     0 Constants parsed
2025-04-24 16:07:03,059 INFO     Grounding start: problema_rueda_pinchada
2025-04-24 16:07:03,059 INFO     Relevance analysis removed 4 facts
2025-04-24 16:07:03,059 INFO     Grounding end: problema_rueda_pinchada
2025-04-24 16:07:03,059 INFO     12 Variables created
2025-04-24 16:07:03,059 INFO     4 Operators created
2025-04-24 16:07:03,061 INFO     Search start: problema_rueda_pi

El paquete [py2pddl](https://github.com/remykarem/py2pddl) facilita la escritura de los ficheros pddl, permitiendo su creación de forma programática. Este paquete no está disponible en el repositorio [PyPI](https://pypi.org/) de paquetes de Python, por lo que su instalación debe realizarse directamente desde GitHub:

`pip install git+https://github.com/remykarem/py2pddl#egg=py2pddl`

Una de las funcionalidades de py2pddl es que comprueba la corrección de los argumentos proporcionados a los predicados en cada uso de estos. Para ello se basa en la extensión de PDDL que establece la forma de especificar tipos y subtipos de objetos:

```
(define
  (domain <nombre_del_dominio>)
  (:requirements :strips :typing)
  (:types
    <nombre_tipo_1> ... <nombre_tipo_M> - object
    <nombre_subtipo_1> ... <nombre_subtipo_N> - <nombre_tipo_1>
    ...
  )
  (:predicates
    (<nombre_del_predicado> <arg_1> - <tipo_arg_1> ... <arg_m> - <tipo_arg_m>)
    ...
  )
  (:action <nombre_de_la_acción>
    :parameters (<arg_1> - <tipo_arg_1> ... <arg_n> - <tipo_arg_n>)
    :precondition (and
      <precondición>
      ...
    )
    :effect (and
      <adición>
      ...
      (not <borrado>)
      ...
    )
  ...
  )
)
```

Para crear el fichero de especificación del dominio con py2pddl hay que definir una clase que herede de la clase `Domain` y en ella usar la función `create_type` para especificar los tipos de objetos y los decoradores `predicate` y `action` para indicar qué métodos son (esquemas de) predicados y cuáles son (esquemas de) acciones. Por ejemplo, en el dominio del mundo de los bloques podríamos especificar que los objetos son de tipo bloque y pypddl se encargaría de realizar todos los chequeos pertinentes.

In [13]:
from py2pddl import Domain, create_type, predicate, action

class MundoBloquesDomain(Domain):  # py2pddl obtiene el nombre del dominio del nombre
                                   # de la clase, pero borrando de él la palabra domain
    Object = create_type("Object")  # Este es el tipo más general posible y se debe
                                    # incluir siempre
    Bloque = create_type("Bloque", Object)  # Establecemos el tipo bloque como un
                                            # subtipo de object
    
    @predicate(Bloque)
    def sobre_la_mesa(self, b):
        """Representa que el bloque b está sobre la mesa"""
    
    @predicate(Bloque, Bloque)
    def sobre(self, b1, b2):
        """Representa que el bloque b1 está sobre el bloque b2"""
    
    @predicate(Bloque)
    def despejado(self, b):
        """Representa que el bloque b no tiene ningún bloque encima"""
    
    @predicate(Bloque)
    def agarrado(self, b):
        """Representa que el brazo robótico ha cogido el bloque b"""
    
    @predicate()
    def brazo_libre(self):
        """Representa que el brazo robótico no tiene cogido ningún bloque"""
    
    @action(Bloque)
    def agarrar(self, b):
        """Representa que el brazo robótico coge el bloque b que está
        sobre la mesa"""
        precondiciones = [self.sobre_la_mesa(b),
                          self.despejado(b),
                          self.brazo_libre()]
        efectos = [self.agarrado(b),
                   ~self.sobre_la_mesa(b),
                   ~self.despejado(b),
                   ~self.brazo_libre()]
        return precondiciones, efectos
    
    @action(Bloque)
    def bajar(self, b):
        """Representa que el brazo robótico deja el bloque b sobre la mesa"""
        precondiciones = [self.agarrado(b)]
        efectos = [self.sobre_la_mesa(b),
                   self.despejado(b),
                   self.brazo_libre(),
                   ~self.agarrado(b)]
        return precondiciones, efectos
    
    @action(Bloque, Bloque)
    def desapilar(self, b1, b2):
        """Representa que el brazo robótico coge el bloque b1 que está
        sobre el bloque b2"""
        precondiciones = [self.sobre(b1, b2),
                          self.despejado(b1),
                          self.brazo_libre()]
        efectos = [self.agarrado(b1),
                   self.despejado(b2),
                   ~self.sobre(b1, b2),
                   ~self.despejado(b1),
                   ~self.brazo_libre()]
        return precondiciones, efectos
    
    @action(Bloque, Bloque)
    def apilar(self, b1, b2):
        """Representa que el brazo robótico deja el bloque b1 sobre
        el bloque b2"""
        precondiciones = [self.agarrado(b1),
                          self.despejado(b2)]
        efectos = [self.sobre(b1, b2),
                   self.despejado(b1),
                   self.brazo_libre(),
                   ~self.agarrado(b1),
                   ~self.despejado(b2)]
        return precondiciones, efectos

Creando una instancia de esta clase y usando el método `generate_domain_pddl` de esa instancia se obtiene un fichero con la especificación PDDL del dominio del problema.

In [14]:
dominio = MundoBloquesDomain()
dominio.generate_domain_pddl(filename="dominio_mundo_bloques")

Domain PDDL written to dominio_mundo_bloques.pddl.


Para crear el fichero de especificación de la instancia del problema con py2pddl hay que definir una clase que herede de la clase del dominio del problema y en ella crear en el método de inicialización los objetos como atributos de la clase, para lo que es útil la función `create_objs`, y usar el decorador `init` sobre el método `init` para especificar el estado inicial y el decorador `goal` sobre el método `goal` para especificar el objetivo.

In [15]:
from py2pddl import goal, init

class MundoBloquesProblem(MundoBloquesDomain):  # py2pddl obtiene el nombre de la instancia
                                                # del nombre de la clase, pero borrando de
                                                # él la palabra problem
    def __init__(self):
        super().__init__()
        self.bloques = MundoBloquesDomain.Bloque.create_objs(["A", "B", "C"])
        # La expresión
        # self.bloques = MundoBloquesDomain.Bloque.create_objs([1, 2, 3], prefix="B")
        # crearía los bloques B1, B2 y B3, a los que luego podríamos referirnos mediante
        # self.bloques[1], self.bloques[2] y self.bloques[3], respectivamente.
    
    @init
    def init(self):
        return [self.sobre(self.bloques["A"], self.bloques["C"]),
                self.despejado(self.bloques["A"]),
                self.sobre_la_mesa(self.bloques["B"]),
                self.despejado(self.bloques["B"]),
                self.sobre_la_mesa(self.bloques["C"]),
                self.brazo_libre()]
    
    @goal
    def goal(self):
        return [self.sobre_la_mesa(self.bloques["A"]),
                self.sobre(self.bloques["B"], self.bloques["A"]),
                self.sobre(self.bloques["C"], self.bloques["B"])]

Creando una instancia de esta clase y usando el método `generate_problem_pddl` de esa instancia se obtiene un fichero con la especificación PDDL de la instancia del problema.

In [16]:
problema = MundoBloquesProblem()
problema.generate_problem_pddl(filename="problema_mundo_bloques")

Problem PDDL written to problema_mundo_bloques.pddl.


Buscamos una solución al problema del mundo de los bloques usando el algoritmo $A^{*}$ y la heurística $h^{m\acute{a}x}$.

In [17]:
!pyperplan -H hmax -s astar dominio_mundo_bloques.pddl problema_mundo_bloques.pddl

2025-04-24 16:12:16,781 INFO     using search: astar_search
2025-04-24 16:12:16,781 INFO     using heuristic: hMaxHeuristic
2025-04-24 16:12:16,781 INFO     Parsing Domain c:\Program Files\practicas IA\Práctica_4\Planificación automática\dominio_mundo_bloques.pddl
2025-04-24 16:12:16,808 INFO     Parsing Problem c:\Program Files\practicas IA\Práctica_4\Planificación automática\problema_mundo_bloques.pddl
2025-04-24 16:12:16,826 INFO     5 Predicates parsed
2025-04-24 16:12:16,826 INFO     4 Actions parsed
2025-04-24 16:12:16,826 INFO     3 Objects parsed
2025-04-24 16:12:16,826 INFO     0 Constants parsed
2025-04-24 16:12:16,826 INFO     Grounding start: mundobloques
2025-04-24 16:12:16,827 INFO     Relevance analysis removed 0 facts
2025-04-24 16:12:16,827 INFO     Grounding end: mundobloques
2025-04-24 16:12:16,827 INFO     19 Variables created
2025-04-24 16:12:16,827 INFO     24 Operators created
2025-04-24 16:12:16,828 INFO     Search start: mundobloques
2025-04-24 16:12:16,828 INF

Aparte del tipado de objetos, existen muchas más extensiones del lenguaje PDDL, como la posibilidad de usar precondiciones negativas en las acciones, la inclusión del predicado de igualdad, la incorporación de variables numéricas, etc. El planificador pyperplan, al estar pensado con un objetivo didáctico, no es compatible con ninguna de ellas. Para la resolución de problemas más complejos de los que se van a considerar en esta práctica debe utilizarse un planificador más completo y eficiente, como puede ser [Fast Downward](https://www.fast-downward.org/).